# 멋진 작사가 만들기
---

이번 프로젝트에서는 49개의 노래가사를 학습하여 스스로 가사를 만드는 인공지는 작사가를 만들어볼 것이다.

# 1. 라이브러리

In [51]:
import os
import glob
import re 
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split


---
# 2. 데이터셋 구성
RNN모델에 들어갈 데이터셋을 구성하는 순서를 요약해보면 다음과 같다
1. 데이터 읽어오기 :학습할 가사내용을 파일에서 읽어오기
2. 문장 필터링 : 불필요한 특수기호나 필요없는 구간은 제외
3. 정규표현식을 이용한 corpus(말뭉치) 생성
4. 토큰화 : tf.keras.preprocessing.text.Tokenizer를 이용해 corpus를 텐서로 변환
5. Dataset 만들기 : tf.data.Dataset.from_tensor_slices()를 이용해 corpus 텐서를 tf.data.Dataset객체로 변환


### 2.1. 데이터 읽어오기
glob 를 활용하여 lyrics 폴더 하위의 모든 txt 파일을 읽어온 후, raw_corpus 리스트에 문장 단위로 저장해보자

In [2]:
txt_file_path = os.getenv('HOME')+ '/aiffel/project/exp04_LyricistAI/data/lyrics/*'
txt_list = glob.glob(txt_file_path)

raw_corpus = []
n_file_count = 0
# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, 'r') as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)
        n_file_count += 1
    
print('파일 개수:', n_file_count)
print('데이터 크기:', len(raw_corpus))              

파일 개수: 49
데이터 크기: 187088


총 49개의 파일에서 18만(187,088)개 이상의 문장을 읽어 raw_corpus 리스트에 설정했다.

### 2.2 문장 필터링
raw_corpus 리스트의 문장에서 필터링할 내용을 미리 걸러주자. 필터링 대상 문장은 모델이 학습하기에 부적절한 내용을 담은 중복된 문장이나 빈문장에 해당한다. 
단, 18만개 이상의 문장을 모두다 들여다 볼수 없으므로 랜덤하게 살펴보도록 하겠다.

In [3]:
raw_corpus[1100:1110]

['Looks like freedom but it feels like death',
 "It's something in between, I guess",
 "It's closing time",
 '(Closing time)',
 '(Closing time)',
 '(Closing time) Yeah I missed you since the place got wrecked',
 'By the winds of change and the weeds of sex',
 'Looks like freedom but it feels like death',
 "It's something in between, I guess",
 "It's closing time Yeah we're drinking and we're dancing"]

In [4]:
raw_corpus[20000:20010]

['When He healed the blind and crippled, did they see?',
 '',
 'Did they speak out against Him, did they dare?',
 'Did they speak out against Him, did they dare?',
 'The multitude wanted to make Him king, put a crown upon His head',
 'Why did He slip away to a quiet place instead?',
 'Did they speak out against Him, did they dare?',
 'Did they speak out against Him, did they dare?',
 '',
 'When He rose from the dead, did they believe?']

In [5]:
raw_corpus[115100:115110]

["A snake is summer's treason,",
 '   And guile is where it goes.',
 '',
 '',
 '',
 '',
 '',
 'XX.',
 '',
 'Could I but ride indefinite,']

문장을 들여다보니 노래가사의 특성상 계속 반복되는 문장이 많은것을 볼수 있다. 그리고 노래사이에 (Closing time) 표현이 들어가있거나 빈문장이 발견되었다. 따라서 아래내용을 전처리 해주어야겠다.
- 빈문장 제외
- 소괄호( ) 문장 제외
- 중복 문장 제외


In [6]:
new_corpus = []
for idx, sentence in enumerate(raw_corpus):
    if len(sentence.strip()) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[0] == "(": continue  # 문장의 시작이 ( 인 문장은 건너뜁니다.    
    
    new_corpus.append(sentence)

new_corpus = list(set(new_corpus)) #중복문장은 set 자료로 형변환하여 제외시킨다
print('필터링후 문장 개수 : ', len(new_corpus))
print(new_corpus[:10])


필터링후 문장 개수 :  116025
['I caress you, let you taste us, just so blissful listen', "I'm on my way to see that girl of mine", 'And when my Mac unloads', "When we kiss And there's a weepy old willow", "He's thirsty when he drinks", '    For want of a nail, the shoe was lost,', 'Hit me to tell me you get off at 10', "I wonder if you'll let us stay with you She was just a little girl, not more than six or seven", 'I’m the one watching you', "I'ma hold you forever,"]


필터링하여 깔끔한 문장이 출력되었다. 18만건이상이던 문장도 11만6000건으로 줄었다. 이제 이문장들을 대상으로 **정규표현식**을 써서 좀 더 다듬어주어야한다.

### 2.3 정규표현식을 이용한 corpus(말뭉치) 생성
 1. 소문자로 바꾸고, 양쪽 공백을 지운다 : sentence.lower().strip()  
 2. a-z가 아닌 모든 문자를 하나의 공백으로 바꾼다 : re.sub
 3. 여러개의 공백은 하나의 공백으로 바꾼다 : re.sub
 4. 다시 양쪽 공백을 지운다 : strip
 5. 문장 시작에는 \<start>, 끝에는 \<end>를 추가

In [32]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()     
    sentence = re.sub(r"[^a-z]+", " ", sentence) #
    sentence = re.sub(r'[" "]+', " ", sentence) 
    sentence = sentence.strip() 
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

In [33]:
last_corpus = []

for sentence in new_corpus:    
    preprocessed_sentence = preprocess_sentence(sentence)
    last_corpus.append(preprocessed_sentence)

# 정제된 결과를 10개만 확인해보죠
last_corpus[:10]

['<start> i caress you let you taste us just so blissful listen <end>',
 '<start> i m on my way to see that girl of mine <end>',
 '<start> and when my mac unloads <end>',
 '<start> when we kiss and there s a weepy old willow <end>',
 '<start> he s thirsty when he drinks <end>',
 '<start> for want of a nail the shoe was lost <end>',
 '<start> hit me to tell me you get off at <end>',
 '<start> i wonder if you ll let us stay with you she was just a little girl not more than six or seven <end>',
 '<start> i m the one watching you <end>',
 '<start> i ma hold you forever <end>']

정규화를 통해 말뭉치(last_corpus)가 완성되었다
- 말뭉치란(코퍼스, corpus) ?
> 분석 대상인 비정형 텍스트 데이터이다. 여러 단어들로 이루어진 문장의 뭉치를 일컫는다.
\<start>와 \<end> 심볼을 넣은 점이 특이한데, 문장의 시작과 끝을 표기해 주어 모델에서 문장의 단위를 알수 있게 한것 같다.

- 단어사전(어휘사전)이란?
> 문장에서 고유한 단어를 뽑아 만든 목록. 토큰화 진행시 단어사전에 없는 단어는 unk로 설정한다.

위의 정제된 문장을 이용해서 단어사전을 만들어보자. 사전을 만든 후 토큰화까지 진행해보겠다. 프로젝트 요구사항에 맞게 단어장의 크기는 12000으로 설정했다.

### 2.4 토큰화 
### 토큰화란?
> 이미지를 학습시킬때 픽셀정보가 수치화되어 있듯이 우리가 준비한 데이터셋도 수치화해주는 작업이 필요하다. 여기서는 텐서플로우의 tf.keras.preprocessing.text.Tokenizer 라이브러리를 사용하여 데이터를 토큰화하여 단어사전을 만들고 데이터를 숫자로 변경해 줄것이다. 숫자로 변환된 데이터를 텐서(tensor) 라고 칭한다고 한다.

### 토큰화 처리 순서
1. 토크나이즈 객체 생성 : Tokenizer 함수  
  - num_words : 단어사전의 단어의 개수 (빈도수가 높은 12000개의 단어를 저장)
  - filters : 토큰화전 필터링 처리하여 따로 설정하지 않음
  - oov_token : 12000 단어에 포함되지 못한 단어는 '\<unk>'로 설정
 > oov란 ? out of vocabulary 즉, train 데이터에서는 등장하지 않았던 단어를 말한다

2. fit_on_texts 
 - 116025개 문장의 last_corpus으로 단어사전 생성
 
3. texts_to_sequences
 - 텍스트를 시퀀스로 변환하기, 즉, 단어별로 인덱스를 부여하는 것이다.
 - tokenizer의 word_index 속성을 확인해보면 딕셔너리 데이터로 해당 텍스트와 숫자가 매칭된 것을 확인할 수 있다.

![이미지](https://codetorial.net/tensorflow/_images/natural_language_processing_in_tensorflow_02.png)

4. pad_sequences
 - 서로 다른 개수의 단어로 이루어진 문장을 같은 길이로 만들어줌
 - padding : 시퀸스 최대길이가 15인데 그보다 짧은 문장의 경우 padding 파라미터로 post를 설정하여 '0' 을 채우도록 하였다
 - maxlen : 시퀀스의 최대길이를 15로 설정
 > **지나치게 긴문장이 있을 경우 다른 문장들이 무의미한 Padding을 가지게 되므로 이번 프로젝트에서는 문장에 포함될 단어의 갯수를 15개로 제한했다.**




In [35]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)   
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15) 
   
    return tensor, tokenizer

tensor, tokenizer = tokenize(last_corpus)

말뭉치의 텍스트가 수치로 변했는지 tokenizer.index_word를 통해 확인해보자

In [36]:
for idx in tokenizer.index_word:
    print(idx, ':', tokenizer.index_word[idx])
    if idx>=10:break
len(tokenizer.index_word)

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : the
6 : you
7 : and
8 : a
9 : to
10 : it


27547

In [37]:
len(tensor), len(tensor[1]), tensor[1]

(116025,
 15,
 array([  2,   4,  19,  18,  12,  84,   9,  61,  16,  80,  17, 212,   3,
          0,   0], dtype=int32))

116025개 문장이 텐서로 생성되었다. 각 텐서의 길이는 15이고 문장의 길이가 짦은경우 뒤쪽에 0으로 패딩이 붙은것도 확인이된다.
tensor[1]를 보면 첫번째 문장 \<start> i m on my way to see that girl of mine \<end>의 각 단어가 숫자로 어떻게 매칭되어있는지 확인 할 수 있다.


### 2.5 Dataset 만들기 
준비된 텐서를 소스와 타겟으로 분리할 차례이다. tf.data.Dataset.from_tensor_slices()를 이용해 텐서를 tf.data.Dataset객체로 변환하고 모델에 학습시키면될 것 이다.


소스 문장(Source Sentence) : 자연어 처리에서 모델의 입력이 되는 문장
 - \<start> this is sample sentence . 문장에 대한 텐서 ->  X_train
    
타겟 문장(Target Sentence) : 정답 역할을 하게 될 모델의 출력 문장
 - this is sample sentence . \<end> 문장에 대한 텐서 -> y_train

 

In [38]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <end> 또는 <pad>가 될것이다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[   2    4 4778    6   60    6  675  133   32   27 9213  426    3    0]
[   4 4778    6   60    6  675  133   32   27 9213  426    3    0    0]


### 훈련 데이터와 평가 데이터를 분리

In [39]:
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, shuffle=True, random_state=2022)

In [40]:
print("train dataset:", enc_train.shape)
print("test dataset:", enc_val.shape)

train dataset: (92820, 14)
test dataset: (23205, 14)


9만개의 학습데이터와 2만 3000건의 검증데이터로 학습을 진행해보겠다

---
# 3. 모델 학습

이프로젝트에서 RNN 모델은 Embedding 레이어, 2개의 LSTM 레이어, 1개의 Dense 레이어로 구성했다.

### RNN 모델에 대한 간략한 이해
입력 데이터
- 단어 사전의 인덱스

Embedding 레이어
- 1차원의 벡터를 2차원으로 바꿔준다
- 워드 벡터는 의미 벡터 공간에서 단어의 추상적 표현(representation)으로 사용된다
- vocab_size : 단어사전의 크기
- embedding_size : 256, 워드 벡터의 차원수, 즉 단어가 추상적으로 표현되는 크기
 -  크기가 2라면 차갑다: [0.0, 1.0], 뜨겁다: [1.0, 0.0], 미지근하다: [0.5, 0.5] 이렇게 단어의 추상적 특징을 수치로 벡터공간에서 표현한다.

LSTM 레이어
- RNN에서 기억 값에 대한 가중치를 제어한다
- hidden_size : 1024, hidden state 의 차원수
- 데이터를 이해하는 신경망 노드의 수
- return_sequences=True : 자신에게 입력된 시퀀스 길이만큼 동일한 시퀀스를 출력(긴문장을 생성해야하므로)
- return_sequences=False : 1개의 벡터만 출력

Dense 레이어
 - 출력값 : 출력층에서 표현되는 단어의 수

In [41]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out


- 손실 함수와 옵티마이저 설정
 - 옵티마이저로 다양한 신경망에서 효과적으로 쓰이는 Adam을 사용했고,
 - 손실함수로는 정수값을 가진 레이블에 대해서 다중 클래스 분류를 수행할때 쓰이는 SparseCategoricalCrossentropy를 썼다.



In [46]:
VOCAB_SIZE = tokenizer.num_words + 1
embedding_size = 256
hidden_size = 1024

model = TextGenerator(VOCAB_SIZE, embedding_size, hidden_size)

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
history = model.fit(enc_train, dec_train, epochs=10, validation_data=(enc_val, dec_val), verbose=2 )


Epoch 1/10
2901/2901 - 111s - loss: 3.4423 - val_loss: 3.2073
Epoch 2/10
2901/2901 - 107s - loss: 3.0604 - val_loss: 3.0720
Epoch 3/10
2901/2901 - 107s - loss: 2.8625 - val_loss: 3.0209
Epoch 4/10
2901/2901 - 107s - loss: 2.6750 - val_loss: 3.0219
Epoch 5/10
2901/2901 - 107s - loss: 2.4864 - val_loss: 3.0575
Epoch 6/10
2901/2901 - 107s - loss: 2.3020 - val_loss: 3.1195
Epoch 7/10
2901/2901 - 107s - loss: 2.1266 - val_loss: 3.1847
Epoch 8/10
2901/2901 - 107s - loss: 1.9601 - val_loss: 3.2580
Epoch 9/10
2901/2901 - 107s - loss: 1.8034 - val_loss: 3.3547
Epoch 10/10
2901/2901 - 107s - loss: 1.6574 - val_loss: 3.4552


학습이 진행되면서 손실이 줄어들었다가 다시늘어나는 과대적합이 나타났다. embedding_size, hidden_size 사이즈를 줄이거나, 늘리는것으로 손실값이 줄어들지 않아 많은 시행착오를 했지만 원했던 결과가 나오지않았다.

In [52]:
VOCAB_SIZE = tokenizer.num_words + 1
embedding_size = 256
hidden_size = 2048

model = TextGenerator(VOCAB_SIZE, embedding_size, hidden_size)

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

early_stopping = EarlyStopping(monitor='val_loss', patience=1)

model.compile(loss=loss, optimizer=optimizer)
history = model.fit(enc_train, dec_train, epochs=10, validation_data=(enc_val, dec_val), verbose=2, callbacks=[early_stopping] )


Epoch 1/10
2901/2901 - 314s - loss: 3.3492 - val_loss: 3.1134
Epoch 2/10
2901/2901 - 312s - loss: 2.9457 - val_loss: 2.9887
Epoch 3/10
2901/2901 - 322s - loss: 2.6664 - val_loss: 2.9592
Epoch 4/10
2901/2901 - 315s - loss: 2.3449 - val_loss: 2.9924


과적합이 계속적으로 나타나 조기종료를 추가하여 일찍 학습은 마쳤다. 원하는 성능까지는 이르지 못했지만 일단 이모델로 테스트를 진행해보았다

---
# 4. 모델 테스트
학습한 모델이 어느정도의 작곡실력을 보여주는지 테스트해 볼 차례이다. 관련함수를 만들고 테스트를 해보자

모델을 테스트하기위해 첫 단어를 \<start> 토큰과 함께 작곡 문장 생성 함수(generate_text)에 넘겨주어야한다.

generate_text 함수의 실행 순서
- 입력받은 단어를 텐서로 변환한다.
- 생성값이 \<end>토큰인지 비교하기위해 미리 \<end>토큰의 텐서를 설정해둔다.
- 예측된 값 중 가장 높은 확률인 word index를 입력텐서뒤에 붙여 test_tensor를 만든다
- 이렇게 예측 단어를 이전 텐서에 붙이는 과정을 반복한다.
- 문장 생성을 마치는 경우
 - \<end>을 예측한경우
 - 문장에 포함된 텐서 길이가 max_len(15)가 되면 종료
 

In [56]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=15):    
    
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]
    
    while True:
        predict = model(test_tensor) 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [60]:
generate_text(model, tokenizer, init_sentence="<start> i love")

'<start> i love you so <end> '

In [57]:
generate_text(model, tokenizer, init_sentence="<start> that")

'<start> that s why i m in love with judas <end> '

In [61]:
generate_text(model, tokenizer, init_sentence="<start> You know")

'<start> you know i m a <unk> <end> '

In [62]:
generate_text(model, tokenizer, init_sentence="<start> Earth")

'<start> earth is gone <end> '

제법 노래로 지어부르기 좋은 가사들을 만드는것 같다. 지구를 넣으니 지구가 사라졌다니 참 창의적(?)인것 같다

---
## 6. 정리

- 비정형데이터중 이미지에 대한 머신러닝 학습만 하다가 텍스트에 대한 학습을 진행하니 생소한 점이 많았다. 이미지의 경우 픽셀에 대한 데이터를 RGB값으로 수치화하는 과정이 쉽게 이해되는 반면에, 텍스트의 경우 텐서(벡터)로 변환하는 과정이 까다롭고 어려웠던 점이 많았다. 단어의 의미를 다차원의 공간에 표시하여 나타낼수있다는 부분도 어려웠지만 비슷한 의미가 가까운 거리에, 반대되는 의미는 먼거리에 위치시킨다는 이론이 와닿았다.


- 전처리시 사용했던 정규화 기법을 적용하기위해 텍스트 데이터에 대한 깊은 이해가 선행되어야할것 같다.


- RNN모델에서 학습할 입력데이터로 이전 레이어의 데이터가 순환적으로 연결되는 구조를 알게되었다. 


- validation loss를 2.2로 만들기까지 레이어도 추가해보고, 히든유닛수나 임베딩사이즈를 조절해보았는데 크게 향상되지않았다. 히든유닛수나 임베딩사이즈를 변경하는것으로 성능이 나아지지않았다.  더 성능을 높이고 싶지만 하이퍼파라미터의 문제보다 근본적인 문제가 있는듯하여  여기까지하고 마치도록 하겠다. 빅데이터까지는 아니지만 많은양의 데이터로 학습후 결과값이 나오기까지 아주 오랜시간이 걸리는 것을 몸소 체험해본 기나긴 프로젝트였다. 



---
# Reference
- 토큰화 : [링크](https://codetorial.net/tensorflow/natural_language_processing_in_tensorflow_01.html)

- NLP : [링크](https://velog.io/@tmddn0311/RNN-tutorial)